# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [3]:
# Gantilah ID file dengan ID dari Google Drive URL
file_id = '1IzpM6HuBhmjC5mXRx3lR5RZYhqkHVr5O'

# Buat URL unduhan langsung
download_url = f'https://drive.google.com/uc?id={file_id}'

# Baca file CSV dari URL
data= pd.read_csv(download_url)

# Tampilkan DataFrame untuk memastikan telah dibaca dengan benar
data.head()

,TransactionAmount,TransactionDate,Location,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,TransactionType,CustomerOccupation,Channel,Cluster
0,14.09,2023-04-11 16:29:14,San Diego,70.0,81.0,1.0,5112.21,2024-11-04 08:08:08,Debit,Engineer,Branch,2
1,376.24,2023-06-27 16:44:19,Houston,68.0,141.0,1.0,13758.91,2024-11-04 08:09:35,Debit,Engineer,Branch,2
2,126.29,2023-07-10 18:16:08,Mesa,19.0,56.0,1.0,1122.35,2024-11-04 08:07:04,Debit,Student,Online,1
3,184.50,2023-05-05 16:32:11,Raleigh,26.0,25.0,1.0,8569.06,2024-11-04 08:09:06,Debit,Student,Online,1
4,13.45,2023-10-16 17:51:24,Atlanta,26.0,198.0,1.0,7429.40,2024-11-04 08:06:39,Debit,Student,Online,1


In [4]:
# Hapus kolom 'TransactionDate', 'PreviousTransactionDate', 'Location', 'TransactionType'
data = data.drop(columns=['TransactionDate', 'PreviousTransactionDate', 'Location', 'TransactionType'])

# Tampilkan DataFrame untuk memastikan kolom telah dihapus
data.head()

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,CustomerOccupation,Channel,Cluster
0,14.09,70.0,81.0,1.0,5112.21,Engineer,Branch,2
1,376.24,68.0,141.0,1.0,13758.91,Engineer,Branch,2
2,126.29,19.0,56.0,1.0,1122.35,Student,Online,1
3,184.50,26.0,25.0,1.0,8569.06,Student,Online,1
4,13.45,26.0,198.0,1.0,7429.40,Student,Online,1


# **Label Encoder**

In [5]:
# Buat instance LabelEncoder
label_encoder = LabelEncoder()

# List kolom kategorikal yang perlu di-encode
categorical_columns = ['CustomerOccupation', 'Channel']

# Encode kolom kategorikal
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column])

# Tampilkan DataFrame untuk memastikan encoding telah diterapkan
data.head()

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,CustomerOccupation,Channel,Cluster
0,14.09,70.0,81.0,1.0,5112.21,0,0,2
1,376.24,68.0,141.0,1.0,13758.91,0,0,2
2,126.29,19.0,56.0,1.0,1122.35,2,1,1
3,184.50,26.0,25.0,1.0,8569.06,2,1,1
4,13.45,26.0,198.0,1.0,7429.40,2,1,1


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [6]:
# Buat instance MinMaxScaler
scaler = MinMaxScaler()

# Normalisasi semua kolom numerik KECUALI kolom Cluster
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
numeric_columns = numeric_columns.drop('Cluster')  # Hindari Cluster dari scaling
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Pisahkan fitur (X) dan target (y)
X = data.drop(columns=['Cluster'])
y = data['Cluster']

# Split data menjadi set pelatihan dan set uji dengan stratifikasi
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

# Cek distribusi label di train dan test set
print("\nDistribusi label di training set:")
print(y_train.value_counts())
print("\nDistribusi label di test set:")
print(y_test.value_counts())


Training set shape: X_train=(2009, 7), y_train=(2009,)
Test set shape: X_test=(503, 7), y_test=(503,)

Distribusi label di training set:
Cluster
2    655
1    525
3    479
0    350
Name: count, dtype: int64

Distribusi label di test set:
Cluster
2    164
1    132
3    120
0     87
Name: count, dtype: int64


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

## **1. Algoritma Random Forest**

Random Forest adalah algoritma machine learning berbasis ensemble yang terdiri dari banyak Decision Tree (pohon keputusan). Algoritma ini bekerja dengan membuat beberapa pohon keputusan (trees) dan menggabungkan hasilnya untuk meningkatkan akurasi dan mengurangi overfitting.Tulis narasi atau penjelasan algoritma yang Anda gunakan.

In [7]:
# Inisialisasi model Random Forest dengan parameter default
rf = RandomForestClassifier(random_state=42)

# Latih model
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
# Prediksi pada Training Data
y_train_pred_rf = rf.predict(X_train)
cm_train_rf = confusion_matrix(y_train, y_train_pred_rf)

print("\n==== Evaluasi Random Forest pada Data Latih (Sebelum Tuning) ====")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred_rf):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_rf, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred_rf, average='weighted'):.4f}")
print(f"F1-Score: {f1_score(y_train, y_train_pred_rf, average='weighted'):.4f}")

# Tampilkan confusion matrix
cm = confusion_matrix(y_train, y_train_pred_rf)
print("\nConfusion Matrix:")
print(cm)



==== Evaluasi Random Forest pada Data Latih (Sebelum Tuning) ====
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000

Confusion Matrix:
[[350   0   0   0]
 [  0 525   0   0]
 [  0   0 655   0]
 [  0   0   0 479]]


## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

In [9]:
# Prediksi menggunakan model pada data uji
y_pred_test = rf.predict(X_test)

# Hitung metrik evaluasi untuk data uji
accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test, average='weighted')
recall = recall_score(y_test, y_pred_test, average='weighted')
f1 = f1_score(y_test, y_pred_test, average='weighted')

# Tampilkan hasil evaluasi pada data uji
print("==== Evaluasi Random Forest pada Data Uji ====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Tampilkan confusion matrix
cm = confusion_matrix(y_test, y_pred_test)
print("\nConfusion Matrix:")
print(cm)

==== Evaluasi Random Forest pada Data Uji ====
Accuracy: 0.8171
Precision: 0.8181
Recall: 0.8171
F1-Score: 0.8176

Confusion Matrix:
[[ 42   0  45   0]
 [  0 132   0   0]
 [ 47   0 117   0]
 [  0   0   0 120]]


## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Definisikan parameter grid untuk tuning
param_dist = {
    'n_estimators': [50, 100, 200, 300, 500],
    'max_depth': [10, 20, 30, 40, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Inisialisasi model Random Forest
rf = RandomForestClassifier(random_state=42)

# Randomized Search
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=20, cv=5,
    scoring='accuracy', n_jobs=-1, random_state=42, verbose=2
)
random_search.fit(X_train, y_train)

# Tampilkan parameter terbaik
print("Best Parameters:", random_search.best_params_)

# Gunakan model terbaik
best_rf = random_search.best_estimator_


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 10}


In [11]:
from sklearn.feature_selection import SelectFromModel

# Gunakan model untuk memilih fitur penting
selector = SelectFromModel(best_rf, threshold='median', prefit=False)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Latih ulang model dengan fitur yang sudah terseleksi
best_rf.fit(X_train_selected, y_train)

# Evaluasi ulang model
y_pred_test = best_rf.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy after Feature Selection:", accuracy)


Accuracy after Feature Selection: 0.8250497017892644


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Lakukan prediksi pada data uji
y_pred_train = best_rf.predict(X_train_selected)
y_pred_test = best_rf.predict(X_test_selected)

# Hitung metrik evaluasi
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')

# Tampilkan hasil evaluasi
print("==== Evaluasi Random Forest setelah Tuning ====")
print(f"Accuracy (Train): {accuracy_train:.4f}")
print(f"Accuracy (Test) : {accuracy_test:.4f}")
print(f"Precision       : {precision_test:.4f}")
print(f"Recall          : {recall_test:.4f}")
print(f"F1-Score        : {f1_test:.4f}")

# Buat confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Tampilkan confusion matrix
cm = confusion_matrix(y_test, y_pred_test)
print("\nConfusion Matrix:")
print(cm)


==== Evaluasi Random Forest setelah Tuning ====
Accuracy (Train): 0.9209
Accuracy (Test) : 0.8250
Precision       : 0.8241
Recall          : 0.8250
F1-Score        : 0.8246

Confusion Matrix:
[[ 42   0  45   0]
 [  0 132   0   0]
 [ 43   0 121   0]
 [  0   0   0 120]]


## Analisis Hasil Model Random Forest

## 1. Perbandingan Sebelum dan Setelah Tuning

- Overfitting sebelum tuning: Model terlalu sempurna pada data latih (Accuracy = 1.0) tetapi turun drastis pada data uji (Accuracy = 0.8171).  
- Setelah tuning, overfitting berkurang: Akurasi data latih turun menjadi 0.9209, tetapi akurasi data uji sedikit meningkat menjadi 0.8250.  

## 2. Identifikasi Kelemahan Model

### Kelemahan Precision dan Recall pada Kelas Tertentu  
Berdasarkan Confusion Matrix, terlihat:  

- Kelas 1 dan Kelas 3 diklasifikasikan dengan baik (tidak ada kesalahan prediksi).  
- Kelas 0 dan Kelas 2 masih memiliki banyak kesalahan klasifikasi, terutama sering tertukar satu sama lain.  

### Overfitting sebelum tuning  

- Model awal terlalu menghafal data latih sehingga performanya menurun di data uji.  
- Setelah tuning, generalisasi model membaik dan lebih adaptif terhadap data uji yang belum pernah dilihat sebelumnya.


### Kesimpulan
 Tuning berhasil mengurangi overfitting, tetapi masih ada ruang untuk perbaikan dengan teknik balancing data atau mencoba model lain.


## **2. Algoritma KNN**

KNN merupakan salah satu algoritma klasifikasi yang paling sederhana dan intuitif dalam machine learning.
Algoritma ini digunakan untuk mengklasifikasikan data baru berdasarkan kedekatannya dengan data yang sudah diberi label dalam dataset pelatihan. KNN sering digunakan karena kemudahannya dalam pemahaman dan implementasi meskipun pada praktiknya, ia dapat menjadi sangat efektif untuk berbagai masalah klasifikasi.

In [15]:
from sklearn.neighbors import KNeighborsClassifier

# Inisialisasi model KNN dengan K=5
knn = KNeighborsClassifier(n_neighbors=5)

# Latih model
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [16]:
# Prediksi pada Data Latih
y_train_pred_knn = knn.predict(X_train)
cm_train_knn = confusion_matrix(y_train, y_train_pred_knn)

print("\n==== Evaluasi KNN pada Data Latih ====")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred_knn):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_knn, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred_knn, average='weighted'):.4f}")
print(f"F1-Score: {f1_score(y_train, y_train_pred_knn, average='weighted'):.4f}")

# Tampilkan confusion matrix
cm = confusion_matrix(y_train, y_train_pred_knn)
print("\nConfusion Matrix:")
print(cm)


==== Evaluasi KNN pada Data Latih ====
Accuracy: 0.8940
Precision: 0.8942
Recall: 0.8940
F1-Score: 0.8941

Confusion Matrix:
[[245   0 105   0]
 [  0 525   0   0]
 [108   0 547   0]
 [  0   0   0 479]]


In [17]:
# Prediksi menggunakan model KNN pada data uji
y_pred_test_knn = knn.predict(X_test)

# Hitung metrik evaluasi untuk data uji
accuracy_knn = accuracy_score(y_test, y_pred_test_knn)
precision_knn = precision_score(y_test, y_pred_test_knn, average='weighted')
recall_knn = recall_score(y_test, y_pred_test_knn, average='weighted')
f1_knn = f1_score(y_test, y_pred_test_knn, average='weighted')

# Tampilkan hasil evaluasi pada data uji
print("==== Evaluasi KNN pada Data Uji ====")
print(f"Accuracy: {accuracy_knn:.4f}")
print(f"Precision: {precision_knn:.4f}")
print(f"Recall: {recall_knn:.4f}")
print(f"F1-Score: {f1_knn:.4f}")

# Tampilkan confusion matrix
cm_knn = confusion_matrix(y_test, y_pred_test_knn)
print("\nConfusion Matrix:")
print(cm_knn)

==== Evaluasi KNN pada Data Uji ====
Accuracy: 0.8290
Precision: 0.8340
Recall: 0.8290
F1-Score: 0.8310

Confusion Matrix:
[[ 49   0  38   0]
 [  0 132   0   0]
 [ 48   0 116   0]
 [  0   0   0 120]]


In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Definisikan rentang hyperparameter
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],  # Coba beberapa nilai K
    'weights': ['uniform', 'distance'],       # Skema pembobotan
    'metric': ['euclidean', 'manhattan']      # Jenis pengukuran jarak
}

# Inisialisasi model KNN
knn = KNeighborsClassifier()

# GridSearchCV untuk mencari parameter terbaik
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Cetak hasil tuning
print("Best Parameters:", grid_search.best_params_)
print("Best Training Accuracy:", grid_search.best_score_)

# Gunakan model terbaik
best_knn = grid_search.best_estimator_



Best Parameters: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}
Best Training Accuracy: 0.8317415416682173


In [20]:
# Evaluasi ulang pada data uji
y_pred_test = best_knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test, average='weighted')
recall = recall_score(y_test, y_pred_test, average='weighted')
f1 = f1_score(y_test, y_pred_test, average='weighted')

print("\n==== Evaluasi KNN setelah Tuning ====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Tampilkan confusion matrix
cm = confusion_matrix(y_test, y_pred_test)
print("\nConfusion Matrix:")
print(cm)



==== Evaluasi KNN setelah Tuning ====
Accuracy: 0.8231
Precision: 0.8309
Recall: 0.8231
F1-Score: 0.8258

Confusion Matrix:
[[ 50   0  37   0]
 [  0 132   0   0]
 [ 52   0 112   0]
 [  0   0   0 120]]


In [21]:
# Evaluasi ulang pada data latih setelah tuning
y_pred_train = best_knn.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')

print("\n==== Evaluasi KNN pada Data Latih setelah Tuning ====")
print(f"Accuracy (Train): {accuracy_train:.4f}")
print(f"Precision       : {precision_train:.4f}")
print(f"Recall          : {recall_train:.4f}")
print(f"F1-Score        : {f1_train:.4f}")

# Tampilkan confusion matrix
cm = confusion_matrix(y_train, y_pred_train)
print("\nConfusion Matrix:")
print(cm)



==== Evaluasi KNN pada Data Latih setelah Tuning ====
Accuracy (Train): 0.9000
Precision       : 0.8998
Recall          : 0.9000
F1-Score        : 0.8998

Confusion Matrix:
[[248   0 102   0]
 [  0 525   0   0]
 [ 99   0 556   0]
 [  0   0   0 479]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

# Analisis Hasil Model KNN

## 1. Perbandingan Sebelum dan Setelah Tuning

- Kinerja sebelum tuning: KNN memiliki akurasi 0.8940 pada data latih dan 0.8290 pada data uji.  
- Setelah tuning, model menjadi sedikit lebih baik pada data latih (0.9000), tetapi sedikit menurun pada data uji (0.8231).  
- Performa stabil antara train dan test, menandakan tidak ada overfitting yang parah.  

## 2. Identifikasi Kelemahan Model

### Precision dan Recall pada Kelas Tertentu  
Berdasarkan Confusion Matrix, terlihat:  

- Kelas 1 dan Kelas 3 memiliki prediksi yang sangat baik (tidak ada kesalahan).  
- Kelas 0 dan Kelas 2 masih memiliki beberapa kesalahan klasifikasi, terutama sering tertukar satu sama lain.  

### Potensi Overfitting atau Underfitting  

- Sebelum tuning, model cukup baik dalam generalisasi karena perbedaan akurasi train dan test kecil.  
- Setelah tuning, perbedaan train-test masih kecil, menunjukkan tidak ada overfitting yang signifikan.  

### Kesimpulan:
Model KNN memiliki performa yang cukup stabil tanpa overfitting, tetapi masih ada ruang untuk peningkatan dengan optimasi parameter atau mencoba metode lain.


# Analisis Perbandingan Model Random Forest vs KNN

## Kinerja pada Data Latih

- Random Forest memiliki akurasi lebih tinggi (92%) dibanding KNN (90%), menunjukkan bahwa model lebih kuat dalam mengenali pola dari data latih.  
- Namun, Random Forest sedikit lebih rentan terhadap overfitting karena performanya turun pada data uji.  

## Kinerja pada Data Uji

- Random Forest dan KNN memiliki akurasi yang hampir sama (0.8250 vs 0.8231).  
- KNN sedikit lebih stabil dalam perbandingan train-test, menunjukkan model tidak terlalu overfitting.  

## Precision, Recall, dan F1-Score

- KNN memiliki precision lebih baik pada data uji (0.8309 vs 0.8241), berarti model lebih akurat dalam mengklasifikasikan positif.  
- Random Forest memiliki recall yang lebih stabil, artinya lebih baik dalam mendeteksi kelas positif secara keseluruhan.  

# Kesimpulan dan Rekomendasi

1. Jika mengutamakan stabilitas model (tidak overfitting), KNN lebih baik karena perbedaannya antara train dan test lebih kecil.  
2. Jika ingin model yang lebih kuat dalam belajar pola data, Random Forest lebih unggul karena memiliki akurasi lebih tinggi pada train data.  
3. Untuk peningkatan performa, dapat dilakukan:  

   - Ensemble Learning (menggabungkan kedua model).  
   - Optimasi hyperparameter lebih lanjut dengan GridSearch atau RandomizedSearch.  
   - Feature engineering lebih lanjut untuk meningkatkan kualitas prediksi.  

